In [1]:
import os
from dotenv import load_dotenv
import google.generativeai as genai

In [2]:
load_dotenv()
GOOGLE_API_KEY = os.environ.get('GOOGLE_API_KEY')
# print(GOOGLE_API_KEY)
genai.configure(api_key=GOOGLE_API_KEY)

In [3]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-pro-exp-0827
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-exp-0827
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.0-flash-exp
models/gemini-exp-1206
models/gemini-exp-1121
models/gemini-exp-1114
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/learnlm-1.5-pro-experimental


In [4]:
model = genai.GenerativeModel('gemini-pro')

In [5]:
prompt = "Tell me a joke about SJU."
response = model.generate_content(prompt)
print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "Why did the SJU student cross the road?\n\nTo get to the other McShain."
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0,
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HATE_SPEECH",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HARASSMENT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
              "probability": "NEGLIGIBLE"
            }
          ]
        }


In [10]:
print(response.text)

Why did the SJU student cross the road?

To get to the library on the other side, because he forgot where he parked his car!


In [35]:
def get_ner(text):
    prompt = f"""
    You will be provided with text delimited by ```.
    INSTRUCTIONS:
    Using name entity recognition (NER) capabilities,
    extract the entities in the text in the following format:
    Item 1: ...
    Item 2: ...
    ...
    Item N: ...

    If the text does not contain entities, then simply write
    <No Entities Found!>

    ```{text}```
    """
    response = model.generate_content(prompt)
    return response

def get_ner_picky(text, examples):
    prompt = f"""
    You will be provided with text delimited by ```.

    # INSTRUCTIONS #
    Extract the entities in the text in the following format:
    Item 1: ...
    Item 2: ...
    ...
    Item N: ...

    If the text does not contain entities, then simply write
    <No Entities Found!>
    ONLY include known entities: individuals, places, brands,
    and products.
    ```{text}```

    # RESTRICTIONS #
    Limit the entities extracted to:
    1. Names of places.
    2. Street names.
    3. City names.
    4. State names or state abbreviations.
    5. Zip codes
    6. Names of persons.

    # EXAMPLES #
    {examples}
    """
    # print(prompt)
    response = model.generate_content(prompt)
    return response

In [36]:
#https://nest.sju.edu/
content = """
LA Fitness
Join anytime from now until July 16th, 2024 and receive a membership at $0 initiation and $29.99 monthly dues per person for you and your family members. If purchased by the deadline, your rate of $29.99 will be locked in until you cancel. Located at 25 E City Ave, Bala Cynwyd, PA 19004, this facility is close to the Hawk Hill campus and is accessible by personal vehicle or by using the Hawk Hill East shuttle that makes a stop at the Bala Shopping Center. Check out their website here or get started with a membership here.
"""
print(get_ner(content).text)

Item 1: LA Fitness
Item 2: July 16th, 2024
Item 3: $0
Item 4: $29.99
Item 5: $29.99
Item 6: 25 E City Ave
Item 7: Bala Cynwyd
Item 8: PA 19004
Item 9: Hawk Hill campus
Item 10: Hawk Hill East shuttle
Item 11: Bala Shopping Center


In [37]:
ner_examples = {"Place": "Comcast Center",
                "Street": "1 Market Street",
                "City": "Philadelphia",
                "State": "PA",
                "Zip": "19004",
                "Person": "Nelson Mendella"}
print(get_ner_picky(content,
                    examples=ner_examples).text)

Item 1: LA Fitness
Item 2: 25 E City Ave
Item 3: Bala Cynwyd
Item 4: PA
Item 5: 19004


In [13]:
content="Hello World!"
print(get_ner(content).text)

<No Entities Found!>



    You will be provided with text delimited by ```.
    
    INSTRUCTIONS:
    extract the entities in the text in the following format:
    Item 1: ...
    Item 2: ...
    ...
    Item N: ...
    
    If the text does not contain entities, then simply write
    <No Entities Found!>
    ONLY include known entities: individuals, places, brands, 
    and products.
    ```Hello World!```
    EXAMPLES: Address: 1 Market Street, Philadelphia, PA 19103Place: Comcast CenterCity: PhiladelphiaPerson: Nelson Mendella
    
<No Entities Found!>


In [107]:
def gen_response(prmpt, max_tokens=50, temperature=1.0):
    config = genai.types.GenerationConfig(
            # Only one candidate for now.
            candidate_count=1,
            #stop_sequences=["x"],
            max_output_tokens=max_tokens,
            temperature=temperature,
        )
    response = model.generate_content(
        prmpt,
        generation_config=config,
    )

    return response.text

def count_tokens(text):
    prmpt = f"""Count the count of tokens for the text in ~~~.
    ~~~
    {text}
    ~~~
    INSTRUCTIONS:
    - Store the words in text in a list (L)
    - From L create a new list of tokens (T)
    - Count of tokens = len(TOKENS)
    - Do not show how the tokens are counted.

    OUTPUT:
    1. T
    2. Total number of tokens in T
    """
    # print(prmpt)
    return gen_response(prmpt, max_tokens=250)

In [111]:
prompt = "Tell me a story about a magic backpack."
output = gen_response(prompt)
print(output, f"... \nWord Count: {len(output.split(' '))}")

Elara wasn't sure what was more peculiar, the dusty old backpack she'd found tucked away in her grandmother's attic, or the fact that it seemed to whisper secrets in her ear. It was a nondescript brown thing, ... 
Word Count: 37


In [120]:
temp = .6
max_tokens = 100
for i in range(5):
    print(i+1, gen_response(prompt, temperature=temp, max_tokens=max_tokens), "\n")

1 Elara, a girl with eyes the color of storm clouds and hair like spun moonlight, lived in a dusty, forgotten corner of the world. Her life was a tapestry of chores and drudgery, woven by her stern aunt who believed in the virtues of hard work and the evils of frivolous dreams. Elara dreamed of adventure, of faraway lands and mythical creatures, but her dreams were always crushed by the weight of reality.

One day, while rummaging through her aunt's 

2 Elara, a wisp of a girl with eyes the color of storm clouds, lived in a village nestled between towering mountains. Her life was simple, filled with the rhythm of the seasons and the earthy smell of the fields. But Elara dreamt of adventure, of faraway lands and shimmering oceans. One day, rummaging through her grandmother's dusty attic, she found it – a worn leather backpack, stitched with silver thread and smelling faintly of lavender.

It wasn't until 

3 Elara, a scrawny girl with eyes the color of storm clouds, had always felt diff

In [113]:
print(f"Text: {output}\n\n", count_tokens(output))

Text: Elara wasn't sure what was more peculiar, the dusty old backpack she'd found tucked away in her grandmother's attic, or the fact that it seemed to whisper secrets in her ear. It was a nondescript brown thing,

 1. ['Elara', 'wasn', "'", 't', 'sure', 'what', 'was', 'more', 'peculiar', ',', 'the', 'dusty', 'old', 'backpack', 'she', "'d", 'found', 'tucked', 'away', 'in', 'her', 'grandmother', "'s", 'attic', ',', 'or', 'the', 'fact', 'that', 'it', 'seemed', 'to', 'whisper', 'secrets', 'in', 'her', 'ear', '.', 'It', 'was', 'a', 'nondescript', 'brown', 'thing', ',', '']
2. 42 



In [75]:
print(output)

The old, worn leather backpack had been passed down through generations in the Miller family. No one knew how it had come into their possession, only that it was rumored to have magical properties. It looked unremarkable, a bit dusty and frayed at


In [ ]:
model = genai.GenerativeModel("gemini-1.5-flash")
chat = model.start_chat(
    history=[
        {"role": "user", "parts": "Hello"},
        {"role": "model", "parts": "Great to meet you. What would you like to know?"},
    ]
)
response = chat.send_message("I have 2 dogs in my house.")
print(response.text)
response = chat.send_message("How many paws are in my house?")
print(response.text)

In [41]:
context = """
You are an assistant that speaks like a radio host.
"""
examples = """
When you change the way you look at things,
the things you look at will change.
"""
prompt = """
Tell me something inspiring.
"""

tokens = 50
messages =  [
{'role':'system', 'content':'You are an assistant that speaks like a radio host.'},
{'role':'user', 'content':'tell me something emotional.'},
{'role':'assistant', 'content':'When you change the way you look at things, \
the things you look at will change.'},
{'role':'assistant', 'content':f"Be realistic and practical. \
Keep your answers to less than {tokens} tokens."},
{'role':'user', 'content':'I expect to be inspired!'}]

response = model.generate_content(messages=messages)

TypeError: GenerativeModel.generate_content() got an unexpected keyword argument 'messages'